# Project 2 — RAG application (LangChain + Chroma + HF LLM)

### 1. Présentation 
Cette cellule sert d'introduction au projet. Elle définit les composants clés :
* **Modèle (LLM) :** Qwen2.5-7B-Instruct (Performance proche de GPT-3.5/4 mais open-source).
* **Framework :** LangChain (pour connecter les données au modèle).
* **Base de données :** ChromaDB (pour stocker la mémoire sémantique des cours).


### 2. Installation des dépendances
**Commande :** `!pip -q install ...`
Cette étape installe les bibliothèques Python nécessaires dans l'environnement (Kaggle/Colab) :
* `langchain` & `langchain-community` : Outils pour construire le RAG.
* `chromadb` : La base de données vectorielle.
* `transformers` & `accelerate` : Pour télécharger et exécuter le modèle Qwen depuis Hugging Face.
* `pypdf`, `python-docx` : Pour extraire le texte des fichiers de cours.

In [1]:
!pip -q install -U \
  langchain langchain-community langchain-core langchain-text-splitters \
  langchain-huggingface chromadb \
  sentence-transformers transformers accelerate \
  pypdf python-docx

print("✅ Install OK")

✅ Install OK


### 3. Importations
**Contenu :** `import os`, `from langchain...`
Chargement des modules nécessaires.
* **Gestion d'erreurs :** Le code utilise des blocs `try/except` pour gérer les différences de versions de LangChain (entre `langchain_huggingface` et `langchain_community`), assurant que le code ne plante pas si l'environnement change.

In [2]:
import os
from typing import List
from pypdf import PdfReader
import docx

# LangChain imports (robustes selon versions)
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embeddings HF
try:
    from langchain_huggingface import HuggingFaceEmbeddings
except Exception:
    # fallback older versions
    from langchain_community.embeddings import HuggingFaceEmbeddings

# Chroma vector store
try:
    from langchain_community.vectorstores import Chroma
except Exception:
    from langchain_chroma import Chroma  
# LLM wrapper
try:
    from langchain_huggingface import HuggingFacePipeline
except Exception:
    from langchain_community.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("✅ Imports OK")

2025-12-16 12:19:02.775140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765887542.797228     352 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765887542.803713     352 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✅ Imports OK


### 4. Ingestion des Données (`load_course_materials`)
Cette fonction est responsable de la lecture des fichiers sources.
1.  **Parcours :** Elle scanne le dossier `DATA_DIR` récursivement.
2.  **Extraction :** Elle détecte le format (.pdf, .docx, .txt) et extrait le texte brut.
3.  **Métadonnées :** Chaque document est tagué avec son nom de fichier et le numéro de page (pour les PDF), ce qui permettra de citer les sources précises dans la réponse finale.

In [3]:
DATA_DIR = "/kaggle/input/mini-data/data"   
def load_course_materials(path: str) -> List[Document]:
    docs: List[Document] = []
    if not os.path.exists(path):
        raise FileNotFoundError(f"Dossier introuvable: {path}")

    for root, _, files in os.walk(path):
        for file in files:
            full = os.path.join(root, file)
            lower = file.lower()

            if lower.endswith(".pdf"):
                try:
                    reader = PdfReader(full)
                    for page_idx, page in enumerate(reader.pages):
                        text = page.extract_text() or ""
                        text = text.strip()
                        if not text:
                            continue
                        docs.append(Document(
                            page_content=text,
                            metadata={"source": full, "filetype": "pdf", "page": page_idx + 1}
                        ))
                except Exception as e:
                    print(f"⚠️ PDF erreur: {full} -> {e}")

            elif lower.endswith(".txt"):
                try:
                    with open(full, "r", encoding="utf-8", errors="ignore") as f:
                        text = f.read().strip()
                    if text:
                        docs.append(Document(
                            page_content=text,
                            metadata={"source": full, "filetype": "txt"}
                        ))
                except Exception as e:
                    print(f"⚠️ TXT erreur: {full} -> {e}")

            elif lower.endswith(".docx"):
                try:
                    d = docx.Document(full)
                    text = "\n".join([p.text for p in d.paragraphs]).strip()
                    if text:
                        docs.append(Document(
                            page_content=text,
                            metadata={"source": full, "filetype": "docx"}
                        ))
                except Exception as e:
                    print(f"⚠️ DOCX erreur: {full} -> {e}")

    return docs

docs_raw = load_course_materials(DATA_DIR)
print(f"✅ Documents chargés: {len(docs_raw)}")

# aperçu
if docs_raw:
    print("Exemple metadata:", docs_raw[0].metadata)
    print("Extrait:", docs_raw[0].page_content[:300])

✅ Documents chargés: 621
Exemple metadata: {'source': '/kaggle/input/mini-data/data/CH1_Big Data Analytics.pdf', 'filetype': 'pdf', 'page': 1}
Extrait: Master Big Data Analytics & Smart Systems (BDSaS)
Big Data Analytics II


### 5. Découpage (Chunking)
**Outil :** `RecursiveCharacterTextSplitter`
Les LLM ont une limite de mémoire (fenêtre de contexte). On ne peut pas leur envoyer tout le cours d'un coup.
* **Chunk Size (900) :** On découpe le texte en blocs de 900 caractères.
* **Overlap (150) :** On garde une superposition de 150 caractères entre les blocs pour ne pas couper une phrase importante au milieu et conserver le contexte.

In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=150,
)

docs = splitter.split_documents(docs_raw)
print("✅ Chunks:", len(docs))

# aperçu d'un chunk
if docs:
    print("Exemple chunk metadata:", docs[0].metadata)
    print("Extrait chunk:", docs[0].page_content[:250])

✅ Chunks: 689
Exemple chunk metadata: {'source': '/kaggle/input/mini-data/data/CH1_Big Data Analytics.pdf', 'filetype': 'pdf', 'page': 1}
Extrait chunk: Master Big Data Analytics & Smart Systems (BDSaS)
Big Data Analytics II


### 6. Indexation Vectorielle (Embeddings & Chroma)
Cette étape transforme le texte en mathématiques pour la recherche.
1.  **Embeddings (`all-MiniLM-L6-v2`) :** Ce petit modèle transforme les phrases en vecteurs numériques (listes de nombres).
2.  **Vector Store (Chroma) :** Les vecteurs sont stockés dans ChromaDB.
3.  **Persistance :** La base est sauvegardée sur le disque (`persist_directory`) pour éviter de tout recalculer à chaque fois.
4.  **Retriever :** La base est convertie en moteur de recherche configuré pour renvoyer les **5 passages** les plus pertinents (`k=5`) pour chaque question.

In [5]:
EMB_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL)
PERSIST_DIR = "./chroma_db_project2"
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=PERSIST_DIR,
)
try:
    vectordb.persist()
except Exception:
    pass

retriever = vectordb.as_retriever(search_kwargs={"k": 5})

print("✅ ChromaDB prêt (persist_directory =", PERSIST_DIR, ")")

✅ ChromaDB prêt (persist_directory = ./chroma_db_project2 )


/tmp/ipykernel_352/1969108273.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


### 7. Chargement du LLM (Qwen)
**Modèle :** `Qwen/Qwen2.5-7B-Instruct`
* Le modèle est chargé en mémoire (GPU si disponible via `device_map="auto"`).
* **Pipeline :** On configure la génération de texte (ex: `temperature`, `max_new_tokens`) pour avoir des réponses précises et non créatives (idéal pour des cours).

In [8]:
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"  

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)

gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=False,
    temperature=None,
    max_new_tokens=300,
    return_full_text=False,
)

llm = HuggingFacePipeline(pipeline=gen_pipe)

print("✅ LLM prêt:", MODEL_NAME)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ LLM prêt: Qwen/Qwen2.5-7B-Instruct


### 8. Construction de la Chaîne RAG
C'est le cerveau de l'application qui relie tout.
1.  **Prompt Engineering :** Le `SYSTEM_MSG` donne des ordres stricts au modèle ("Tu es un assistant utile...", "Utilise uniquement le contexte fourni").
2.  **Formatage :** La fonction `format_docs` prépare les extraits de cours trouvés en ajoutant `[SOURCE: ...]` devant.
3.  **Template Chat :** On utilise le format spécifique attendu par Qwen (`<|im_start|>system...`) pour qu'il comprenne bien son rôle.
4.  **La Chaîne (LCEL) :** La syntaxe `retriever | format | prompt | llm` crée le flux automatique : *Question -> Recherche -> Prompt -> Génération*.

In [9]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Message système (string multi-lignes correcte)
SYSTEM_MSG = """Tu es un assistant qui répond UNIQUEMENT à partir du contexte.

RÈGLES :
- Si la réponse n'est pas dans le contexte → réponds EXACTEMENT :
  "Je ne sais pas, ce n'est pas indiqué dans les documents."
- Ne fais AUCUNE déduction.
- Réponds toujours en français.
"""

def format_docs(docs: List[Document]) -> str:
    """Concatène les documents récupérés avec leurs sources"""
    blocks = []
    for d in docs:
        src = d.metadata.get("source", "unknown")
        page = d.metadata.get("page", None)

        if page is not None:
            header = f"[SOURCE: {src} | page {page}]"
        else:
            header = f"[SOURCE: {src}]"

        blocks.append(header + "\n" + d.page_content)

    return "\n\n".join(blocks)

def build_qwen_prompt(inputs: dict) -> str:
    """Construit le prompt compatible avec le chat template Qwen"""
    context = inputs["context"]
    question = inputs["question"]

    user_msg = (
        f"Contexte :\n{context}\n\n"
        f"Question :\n{question}\n\n"
        f"Réponse :"
    )

    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user", "content": user_msg},
    ]

    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

# Chaîne RAG LangChain (LCEL)
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_qwen_prompt)
    | llm
    | StrOutputParser()
)

print("✅ RAG chain prête")


✅ RAG chain prête


### 9. Exécution et Test
**Fonctions :** `ask(question)` et `ask1(question)`
* Ces fonctions envoient une question à la chaîne RAG.
* `ask` : Affiche la réponse + les sources détaillées (nom du fichier, page).
* `ask1` : Affiche uniquement la réponse générée.

In [10]:
def ask(question: str, k: int = 5):
    try:
        docs = retriever.get_relevant_documents(question)
    except Exception:
        docs = retriever.invoke(question)

    answer = rag_chain.invoke(question)

    print("QUESTION:", question)
    print("\nRÉPONSE:\n", answer)

    print("\nSOURCES (top-k):")
    for i, d in enumerate(docs[:k], 1):
        src = d.metadata.get("source", "unknown")
        page = d.metadata.get("page", "")
        if page:
            print(f"{i}. {src} (page {page})")
        else:
            print(f"{i}. {src}")

In [13]:
def ask1(question: str):
    answer = rag_chain.invoke(question)

    print("QUESTION:", question)
    print("\nRÉPONSE:\n", answer)


In [17]:
ask("What is Memory-based Collaborative Filtering ?")

QUESTION: What is Memory-based Collaborative Filtering ?

RÉPONSE:
 Memory-based Collaborative Filtering, également appelé l'approche de voisinage, utilise directement la base de données entière d'utilisateurs et d'items (la matrice d'utilité) pour générer des prédictions, sans construire de modèle. Cette approche inclut à la fois les algorithmes basés sur les utilisateurs et ceux basés sur les items. En essentiel, un tel algorithme trouve explicitement des utilisateurs ou des items similaires (appelés voisins) et utilise ces derniers pour prédire les préférences de l'utilisateur cible. Il utilise des classifieurs de voisins les plus proches pour prédire les notes des utilisateurs ou leur propension à acheter en mesurant la corrélation entre le profil de l'utilisateur cible (qui peut être un ensemble de notes d'items ou un ensemble d'items visités ou achetés) et les profils d'autres utilisateurs.

SOURCES (top-k):
1. /kaggle/input/mini-data/data/3 Memory-based Collaborative Filtering-3

In [14]:
ask1("What is Bitcoin ?")

QUESTION: What is Bitcoin ?

RÉPONSE:
 Bitcoin est le nom d'un protocole, d'un réseau peer-to-peer et d'une innovation informatique distribuée. C'est également la première application d'une invention qui représente l'aboutissement de décennies de recherche en cryptographie et en systèmes distribués. Il s'agit d'un ensemble de concepts et de technologies constituant la base d'un écosystème de monnaie numérique, comprenant quatre innovations clés : un réseau peer-to-peer décentralisé (le protocole Bitcoin), un registre public des transactions (la blockchain), un ensemble de règles pour la validation indépendante des transactions et l'émission de devises (règles de consensus), et un mécanisme pour parvenir à un consensus décentralisé mondial sur la blockchain valide (algorithme Proof-of-Work).
